In [ ]:
import numpy as np
from collections import deque
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import pickle

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

#Игра

In [ ]:
# Игра крестики-нолики
class TicTacToe:
    def __init__(self, player_1, player_2, board_size=3, win_size=3):
        self.players = {-1: player_1,
                         1: player_2}

        self.wins = {player_1.name: 0,
                     player_2.name: 0}

        self.board_size=board_size
        self.win_size = win_size
        self._kernel = self._create_kernel()


    # Создает ядро свертки для расчета побед
    def _create_kernel(self):
        kernel = np.zeros((2 * self.win_size + 2, self.win_size, self.win_size))
        for i in range(self.win_size):
            kernel[i, i, :] = np.ones(self.win_size)
        for i in range(self.win_size, 2 * self.win_size):
            kernel[i, :, i - self.win_size] = np.ones(self.win_size).T
        kernel[2 * self.win_size] = np.eye(self.win_size)
        kernel[2 * self.win_size + 1] = np.fliplr(np.eye(self.win_size))
        return kernel


    # Проверяет победы для состояний states, в кот. ходы были совершены игроками turns, turn={-1, 1}
    def _test_win(self, state, turn):
        rows, cols, w_size = *state.shape, self.win_size
        expanded_states = np.lib.stride_tricks.as_strided(
            state,
            shape=(rows - w_size + 1, cols - w_size + 1, w_size, w_size),
            strides=(*state.strides, *state.strides),
            writeable=False,
        )
        feature_map = np.einsum('xyij,sij->sxy', expanded_states, self._kernel)
        return -turn * (feature_map == turn * w_size).any().astype(int)


    # Проигрывание нескольких полных эпизодов
    def play(self, num_games=1, visualize=False):
        transitions = []
        for t in range(num_games):
            next_turn = turn = -1
            state = (np.zeros((self.board_size, self.board_size)), turn) # Начальное состояние игры. state = (state2d, turn)
            if visualize:
                self.visualize_state(state, turn)
            while(next_turn != 0):
                state_2d, turn = state
                current_player = self.players[turn]
                action = current_player.get_action(state)
                next_state_2d, next_turn, reward = self.play_turn(state, action)
                transitions.append((turn * state_2d, action, reward, -turn * next_state_2d, next_turn == 0))   #state, action, reward, new_state, done
                if visualize:
                    self.visualize_state((next_state_2d, next_turn), turn)
                if next_turn == 0:
                    if visualize:
                        if (reward == 0): print('Ничья!\n')
                        else: print(f'Победа ({self.players[reward * turn].name})!\n')
                    if reward != 0:
                        self.wins[self.players[reward * turn].name] += 1
                    self.players = {-1: self.players[1], 1: self.players[-1]}
                state = next_state_2d, next_turn
        return transitions


    # Выполнение хода и проверка на некорректный ход (проигрышь) / выигрыш / ничью
    def play_turn(self, state, action): # next_state2d, next_turn, reward
        state2d, turn = state
        next_state2d = state2d.copy()

        # Проверка корректности хода
        if (state2d[(action)] != 0):
            return next_state2d, 0, -1        # Игрок проиграл (# next_turn == 0 => Игра окончена)

        # Совершение хода
        next_state2d[action] = turn

        # Проверка победы
        if self._test_win(next_state2d, turn):
            return next_state2d, 0, 1         # Текущий игрок побеждает (next_turn == 0 => Игра окончена)

        # Проверка ничьи
        if (next_state2d != 0).all():
            return next_state2d, 0, 0         # Ничья (next_turn == 0 => Игра окончена)

        # Инчае, ход следующего игрока
        return next_state2d, -turn, 0         # next_turn == -turn => Смена хода


    # Выводит на экран состояние игры после хода игрока
    @staticmethod
    def visualize_state(next_state, turn):
        next_state2d, next_turn = next_state
        print(f"player {turn}'s turn:")
        if (next_state2d == 0).all() and turn == 0:
            print("[invalid state]\n\n")
        else:
            print(str(next_state2d)
                  .replace(".", "")
                  .replace("[[", "")
                  .replace(" [", "")
                  .replace("]]", "")
                  .replace("]", "")
                  .replace("-0", " .")
                  .replace("0", ".")
                  .replace("-1", " X")
                  .replace("1", "O")
            )


    @staticmethod
    def print_transitions(transitions):
        states, actions, rewards, next_states, dones = zip(*transitions)
        for i in np.arange(len(states)):
            print("\033[31m{}.".format(i + 1), '\033[30m')
            TicTacToe.visualize_state((next_states[i], -1), 1)
            print('\naction = ', actions[i] + np.array([1, 1]), end='\n')
            print('reward = ', rewards[i], end='\n')
            if (dones[i]): print('Игра окончена', end='\n\n')
            else: print('Игра продолжается', end='\n\n')

#Игроки

In [ ]:
class Human:
    def __init__(self, name='Human'):
        self.name = name

    def get_action(self, state):
        state2d, turn = state
        print('Введите ваш ход (Строка, столбец)')
        row, col = map(int, input().split())
        while (state2d[row - 1, col - 1] != 0):
            print('Клетка занята!')
            print('Введите ваш ход (Строка, столбец)')
            row, col = map(int, input().split())
        return row - 1, col - 1

In [ ]:
class Random:
    def __init__(self, name='Random'):
        self.name = name

    def get_action(self, state):
        state2d, turn = state
        zero_idxs = np.argwhere(state2d == 0)
        return tuple(zero_idxs[np.random.randint(len(zero_idxs))])

In [ ]:
# Игрок Рандом с преимуществами:
# 1. Если есть возможность выиграть за один ход, он делает это
# 2. Выбирает случайный ход из множества допустимых
class WinRandom:
    def __init__(self, board_size=3, win_size=3, name='WinRandom'):
        self.name = name
        self.board_size = board_size
        self.win_size = win_size
        self._kernel = self._create_kernel()


    # Создает ядро свертки для расчета потенциальных побед
    def _create_kernel(self):
        kernel = np.zeros((2 * self.win_size + 2, self.win_size, self.win_size))
        for i in range(self.win_size):
            kernel[i, i, :] = np.ones(self.win_size)
        for i in range(self.win_size, 2 * self.win_size):
            kernel[i, :, i - self.win_size] = np.ones(self.win_size).T
        kernel[2 * self.win_size] = np.eye(self.win_size)
        kernel[2 * self.win_size + 1] = np.fliplr(np.eye(self.win_size))
        return kernel


    def get_action(self, state):
        state2d, turn = state
        rows, cols, w_size = *state2d.shape, self.win_size

        expanded_states = np.lib.stride_tricks.as_strided(
            state2d,
            shape=(rows - w_size + 1, cols - w_size + 1, w_size, w_size),
            strides=(*state2d.strides, *state2d.strides),
            writeable=False,
        )
        feature_map = np.einsum('xyij,sij->sxy', expanded_states, self._kernel)
        wins = np.array(np.where(turn * feature_map == w_size - 1))

        if wins.shape[1] > 0:
            K, I, J = wins[:, 0]
            indxs = np.where(np.logical_and((self._kernel[K] == 1), (state2d[I: I + w_size, J: J + w_size] == 0)))
            action = np.array(indxs)[:, 0] + [I, J]
        else:
            zero_idxs = np.argwhere(state2d == 0)
            action = zero_idxs[np.random.randint(len(zero_idxs))]

        return tuple(action)

In [ ]:
# Игрок Рандом с преимуществами:
# 1. Если есть возможность выиграть за один ход, он делает это
# 2. Если у соперника есть возможность выиграть в следующем ходу, он блокирует этот ход
# 3. Иначе, выбирает случайный ход из множества допустимых
class RandomWinPlus:
    def __init__(self, board_size=3, win_size=3, name='RandomWinPlus'):
        self.name = name
        self.board_size = board_size
        self.win_size = win_size
        self._kernel = self._create_kernel()


    # Создает ядро свертки для расчета потенциальных побед
    def _create_kernel(self):
        kernel = np.zeros((2 * self.win_size + 2, self.win_size, self.win_size))
        for i in range(self.win_size):
            kernel[i, i, :] = np.ones(self.win_size)
        for i in range(self.win_size, 2 * self.win_size):
            kernel[i, :, i - self.win_size] = np.ones(self.win_size).T
        kernel[2 * self.win_size] = np.eye(self.win_size)
        kernel[2 * self.win_size + 1] = np.fliplr(np.eye(self.win_size))
        return kernel


    def get_action(self, state):
        state2d, turn = state
        rows, cols, w_size = *state2d.shape, self.win_size

        expanded_states = np.lib.stride_tricks.as_strided(
            state2d,
            shape=(rows - w_size + 1, cols - w_size + 1, w_size, w_size),
            strides=(*state2d.strides, *state2d.strides),
            writeable=False,
        )
        feature_map = np.einsum('xyij,sij->sxy', expanded_states, self._kernel)

        wins = np.array(np.where(turn * feature_map == w_size - 1))
        if wins.shape[1] > 0:
            K, I, J = wins[:, 0]
            indxs = np.where(np.logical_and((self._kernel[K] == 1), (state2d[I: I + w_size, J: J + w_size] == 0)))
            return tuple(np.array(indxs)[:, 0] + [I, J])

        defenses = np.array(np.where(-turn * feature_map == w_size - 1))
        if defenses.shape[1] > 0:
            K, I, J = defenses[:, 0]
            indxs = np.where(np.logical_and((self._kernel[K] == 1), (state2d[I: I + w_size, J: J + w_size] == 0)))
            return tuple(np.array(indxs)[:, 0] + [I, J])

        zero_idxs = np.argwhere(state2d == 0)
        return tuple(zero_idxs[np.random.randint(len(zero_idxs))])

In [ ]:
class DQNAgent(nn.Module):
    def __init__(self, epsilon=0, name='DQNAgent'):
        super().__init__()

        self.name = name
        self.epsilon = epsilon
        self.n_channels = 3

        self.network = nn.Sequential(
            nn.Conv2d(self.n_channels, 64, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.Conv2d(64, 256, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.Conv2d(256, 1, kernel_size=(3, 3), padding='same')
        )

    def forward(self, x):
        x = torch.stack([x == 1, x == -1, x == 0], axis=1).float()
        return self.network(x).squeeze(1)

    def greedy_action(self, state, device=device):
        state2d, turn = state
        state_t = torch.FloatTensor(turn * state2d).unsqueeze(0).to(device)
        q_values = self.forward(state_t).squeeze(0).detach().cpu().numpy()
        # q_values[state2d != 0] = -float("Inf") # Маскирование
        return np.unravel_index(q_values.argmax(), q_values.shape)

    def random_action(self, state):
        state2d, turn = state
        zero_idxs = np.argwhere(state2d == 0)
        return tuple(zero_idxs[np.random.randint(len(zero_idxs))])

    def get_action(self, state):
        if random.random() < self.epsilon:
            action = self.random_action(state)
        else:
            action = self.greedy_action(state)
        return action

#Функции и гиперпараметры для обучения

In [ ]:
class ReplayBuffer(object):
    def __init__(self, size):
        self._storage = deque(maxlen=size)

    def __len__(self):
        return len(self._storage)

    def add(self, transition):
        self._storage.append(transition)

    def sample(self, batch_size):
        batch = random.sample(self._storage, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)

In [ ]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed);

In [ ]:
board_size = 5
win_size = 4

In [ ]:
# Гиперпараметры метода DQN

batch_size = 128
total_steps = 60_000

decay_steps = 40_000
init_epsilon = 1
final_epsilon = 0.1

loss_freq = 100
refresh_target_network_freq = 100

eval_freq = 1000
n_eval_games = 50

max_grad_norm = 50

gamma = 1.0

In [ ]:
agent = DQNAgent(init_epsilon).to(device)

target_network = DQNAgent(init_epsilon).to(device)
target_network.load_state_dict(agent.state_dict())

optimizer = torch.optim.Adam(agent.parameters(), lr=1e-4)
exp_replay = ReplayBuffer(16_000)

In [ ]:
sum([p.numel() for p in agent.parameters()])

741889

In [ ]:
# Возвращает temporal difference loss
def compute_td_loss(states, actions, rewards, next_states, dones,
                    agent, target_network, gamma=0.9, device=device):

    states = torch.tensor(states, device=device, dtype=torch.float32)                # shape: [batch_size, state_dim]
    actions = torch.tensor(actions, device=device, dtype=torch.int64)                # shape: [batch_size]
    rewards = torch.tensor(rewards, device=device, dtype=torch.float32)              # shape: [batch_size]
    next_states = torch.tensor(next_states, device=device, dtype=torch.float32)      # shape: [batch_size, state_dim]
    dones = torch.tensor(dones, device=device, dtype=torch.int64)                    # shape: [batch_size]

    predicted_qvalues = agent(states)                                                # shape: [batch_size, n_actions]
    predicted_next_qvalues = target_network(next_states)                             # shape: [batch_size, n_actions]
    predicted_qvalues_for_actions = predicted_qvalues[range(len(actions)), actions[:, 0], actions[:, 1]]  # shape: [batch_size]
    next_state_values = predicted_next_qvalues.view(dones.shape[0], -1).max(axis=1).values
    target_qvalues_for_actions = rewards - (1 - dones) * gamma * next_state_values
    return torch.mean((predicted_qvalues_for_actions - target_qvalues_for_actions.detach()) ** 2)  #loss

# Рассчитывает epsilon на текущем шаге step
def linear_decay(init_epsilon, final_epsilon, step, decay_steps):
    return max(init_epsilon - step * (init_epsilon - final_epsilon) / decay_steps, final_epsilon)

# Обучение

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
game = TicTacToe(agent, WinRandom(), board_size=board_size, win_size=win_size)

In [ ]:
PATH = f'/content/drive/MyDrive/TicTacToe/'

loss = None
loss_values = []
reward_values = []

In [ ]:
with open('out2.txt', 'w') as f:
  for t in range(total_steps):
    print(f't = {t}. Ход {game.players[-1].name}', file=f)

    state = (np.zeros((board_size, board_size)), -1) # Начальное состояние игры. state = (state_2d, turn)
    turn = next_turn = -1

    while(next_turn != 0):
        current_player = game.players[turn]
        if current_player.name == 'DQNAgent':
            agent.epsilon = linear_decay(init_epsilon, final_epsilon, t, decay_steps)

        action = agent.get_action(state)
        print(action, file=f)
        next_state_2d, next_turn, reward = game.play_turn(state, action)

        if next_turn == 0:
            if (reward == 0): print('Ничья!\n', file=f)
            else: print(f'Победа ({game.players[reward * turn].name})!\n', file=f)
            game.players = {-1: game.players[1], 1: game.players[-1]}

        state_2d, turn = state
        exp_replay.add((turn * state_2d, action, reward, next_turn * next_state_2d, next_turn == 0)) #state, action, reward, new_state, done

        # Обучение на минибатче
        if len(exp_replay) >= batch_size:
            states, actions, rewards, next_states, dones = exp_replay.sample(batch_size)
            loss = compute_td_loss(states, actions, rewards, next_states, dones, agent, target_network)
            loss.backward()
            grad_norm = nn.utils.clip_grad_norm_(agent.parameters(), max_grad_norm)
            optimizer.step()
            optimizer.zero_grad()

        state = next_state_2d, next_turn

    # Каждые refresh_target_network_freq обновляются веса target сети
    if t % refresh_target_network_freq == 0:
        target_network.load_state_dict(agent.state_dict())

    # Вывод лосса  с заданной частотой
    if t % loss_freq == 0 and t > 0:
        loss_values.append(loss)
        print(f"t = {str(t):5}\t loss = {loss}\t eps = {round(agent.epsilon, 4)}")

    # Вывод награды с заданной частотой
    if t % eval_freq == 0:
        agent.epsilon = 0
        eval_game = TicTacToe(agent, WinRandom(), board_size=board_size, win_size=win_size)
        eval_game.play(n_eval_games)
        mean_reward = (eval_game.wins['DQNAgent'] - eval_game.wins['WinRandom']) / n_eval_games
        reward_values.append(mean_reward)
        print(f"t = {str(t):5}\t reward = {round(mean_reward, 4)}\n")

        torch.save(agent.state_dict(), PATH + f'model_{t}')
        torch.save(optimizer.state_dict(), PATH + f'opt_{t}')

t = 0    	 reward = -1.0

t = 100  	 loss = 0.0009138107998296618	 eps = 0.9976
t = 200  	 loss = 0.019590014591813087	 eps = 0.9951
t = 300  	 loss = 0.0026408215053379536	 eps = 0.9927
t = 400  	 loss = 0.002870304509997368	 eps = 0.9902
t = 500  	 loss = 0.003184820991009474	 eps = 0.9878
t = 600  	 loss = 0.0030174986459314823	 eps = 0.9853
t = 700  	 loss = 0.002803496550768614	 eps = 0.9829
t = 800  	 loss = 0.0035816256422549486	 eps = 0.9804
t = 900  	 loss = 0.0038023940287530422	 eps = 0.9779
t = 1000 	 loss = 0.007283843122422695	 eps = 0.9755
t = 1000 	 reward = 0.28

t = 1100 	 loss = 0.0025569952558726072	 eps = 0.973
t = 1200 	 loss = 0.0021268613636493683	 eps = 0.9706
t = 1300 	 loss = 0.0023099936079233885	 eps = 0.9681
t = 1400 	 loss = 0.008836787194013596	 eps = 0.9657
t = 1500 	 loss = 0.001790046226233244	 eps = 0.9633
t = 1600 	 loss = 0.0012909355573356152	 eps = 0.9608
t = 1700 	 loss = 0.0019846735522150993	 eps = 0.9584
t = 1800 	 loss = 0.001126698334701359

KeyboardInterrupt: 

In [ ]:
agent.load_state_dict(torch.load(PATH + 'model_40000'))
optimizer.load_state_dict(torch.load(PATH + 'opt_40000'))

In [ ]:
with open('out3.txt', 'w') as f:
  for t in range(40_000, total_steps):
    print(f't = {t}. Ход {game.players[-1].name}', file=f)

    state = (np.zeros((board_size, board_size)), -1) # Начальное состояние игры. state = (state_2d, turn)
    turn = next_turn = -1

    while(next_turn != 0):
        current_player = game.players[turn]
        if current_player.name == 'DQNAgent':
            agent.epsilon = linear_decay(init_epsilon, final_epsilon, t, decay_steps)

        action = agent.get_action(state)
        print(action, file=f)
        next_state_2d, next_turn, reward = game.play_turn(state, action)

        if next_turn == 0:
            if (reward == 0): print('Ничья!\n', file=f)
            else: print(f'Победа ({game.players[reward * turn].name})!\n', file=f)
            game.players = {-1: game.players[1], 1: game.players[-1]}

        state_2d, turn = state
        exp_replay.add((turn * state_2d, action, reward, next_turn * next_state_2d, next_turn == 0)) #state, action, reward, new_state, done

        # Обучение на минибатче
        if len(exp_replay) >= batch_size:
            states, actions, rewards, next_states, dones = exp_replay.sample(batch_size)
            loss = compute_td_loss(states, actions, rewards, next_states, dones, agent, target_network)
            loss.backward()
            grad_norm = nn.utils.clip_grad_norm_(agent.parameters(), max_grad_norm)
            optimizer.step()
            optimizer.zero_grad()

        state = next_state_2d, next_turn

    # Каждые refresh_target_network_freq обновляются веса target сети
    if t % refresh_target_network_freq == 0:
        target_network.load_state_dict(agent.state_dict())

    # Вывод лосса с заданной частотой
    if t % loss_freq == 0 and t > 0:
        loss_values.append(loss)
        print(f"t = {str(t):5}\t loss = {loss}\t eps = {round(agent.epsilon, 4)}")

    # Вывод награды с заданной частотой
    if t % eval_freq == 0:
        agent.epsilon = 0
        eval_game = TicTacToe(agent, WinRandom(), board_size=board_size, win_size=win_size)
        eval_game.play(n_eval_games)
        mean_reward = (eval_game.wins['DQNAgent'] - eval_game.wins['WinRandom']) / n_eval_games
        reward_values.append(mean_reward)
        print(f"t = {str(t):5}\t reward = {round(mean_reward, 4)}\n")

        torch.save(agent.state_dict(), PATH + f'model_{t}')
        torch.save(optimizer.state_dict(), PATH + f'opt_{t}')

t = 40000	 loss = 0.00011160914436914027	 eps = 0.1
t = 40000	 reward = 0.96

t = 40100	 loss = 0.0002448922023177147	 eps = 0.1
t = 40200	 loss = 1.1537428690644447e-05	 eps = 0.1
t = 40300	 loss = 0.0001940709917107597	 eps = 0.1
t = 40400	 loss = 0.00015100810560397804	 eps = 0.1
t = 40500	 loss = 0.00010194587957812473	 eps = 0.1
t = 40600	 loss = 0.00015467706543859094	 eps = 0.1
t = 40700	 loss = 6.996030424488708e-05	 eps = 0.1
t = 40800	 loss = 0.00017945987929124385	 eps = 0.1
t = 40900	 loss = 0.00017210238729603589	 eps = 0.1
t = 41000	 loss = 0.0002223863557446748	 eps = 0.1
t = 41000	 reward = 0.8

t = 41100	 loss = 6.522104376927018e-05	 eps = 0.1
t = 41200	 loss = 0.00038907479029148817	 eps = 0.1
t = 41300	 loss = 0.00032318499870598316	 eps = 0.1
t = 41400	 loss = 9.38091761781834e-05	 eps = 0.1
t = 41500	 loss = 0.00013921585923526436	 eps = 0.1
t = 41600	 loss = 0.00014017298235557973	 eps = 0.1
t = 41700	 loss = 7.385536446236074e-05	 eps = 0.1
t = 41800	 loss = 0.0

#Тестирование обученных моделей

In [ ]:
# Сравнение обученных моделей

for i in range(61):
    agent.load_state_dict(torch.load(f'/content/drive/MyDrive/TicTacToe/model_{i * 1000}', map_location=torch.device('cpu')))
    agent.epsilon = 0
    eval_game = TicTacToe(agent, RandomWinPlus(), board_size=board_size, win_size=win_size)
    eval_game.play(100)
    print(i, eval_game.wins)

0 {'DQNAgent': 0, 'RandomWinPlus': 100}
1 {'DQNAgent': 20, 'RandomWinPlus': 80}
2 {'DQNAgent': 24, 'RandomWinPlus': 76}
3 {'DQNAgent': 32, 'RandomWinPlus': 68}
4 {'DQNAgent': 66, 'RandomWinPlus': 34}
5 {'DQNAgent': 61, 'RandomWinPlus': 39}
6 {'DQNAgent': 57, 'RandomWinPlus': 43}
7 {'DQNAgent': 70, 'RandomWinPlus': 30}
8 {'DQNAgent': 74, 'RandomWinPlus': 26}
9 {'DQNAgent': 67, 'RandomWinPlus': 33}
10 {'DQNAgent': 72, 'RandomWinPlus': 28}
11 {'DQNAgent': 73, 'RandomWinPlus': 27}
12 {'DQNAgent': 75, 'RandomWinPlus': 25}
13 {'DQNAgent': 84, 'RandomWinPlus': 16}
14 {'DQNAgent': 88, 'RandomWinPlus': 12}
15 {'DQNAgent': 85, 'RandomWinPlus': 15}
16 {'DQNAgent': 86, 'RandomWinPlus': 14}
17 {'DQNAgent': 87, 'RandomWinPlus': 13}
18 {'DQNAgent': 84, 'RandomWinPlus': 16}
19 {'DQNAgent': 96, 'RandomWinPlus': 4}
20 {'DQNAgent': 95, 'RandomWinPlus': 5}
21 {'DQNAgent': 80, 'RandomWinPlus': 20}
22 {'DQNAgent': 85, 'RandomWinPlus': 15}
23 {'DQNAgent': 91, 'RandomWinPlus': 9}
24 {'DQNAgent': 92, 'RandomWi

In [ ]:
# Загрузка лучшей модели
agent.load_state_dict(torch.load('/content/drive/MyDrive/TicTacToe/model_37000', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
agent.epsilon = 0
test_game = TicTacToe(agent, Human(), board_size=board_size, win_size=win_size)
test_game.play(10, True)

player -1's turn:
. . . . .
. . . . .
. . . . .
. . . . .
. . . . .
player -1's turn:
 .  .  .  .  .
 .  .  .  .  .
 .  .  X  .  .
 .  .  .  .  .
 .  .  .  .  .
Введите ваш ход (Строка, столбец)
1 1
player 1's turn:
 O  .  .  .  .
 .  .  .  .  .
 .  .  X  .  .
 .  .  .  .  .
 .  .  .  .  .
player -1's turn:
 O  .  .  .  .
 .  .  .  X  .
 .  .  X  .  .
 .  .  .  .  .
 .  .  .  .  .
Введите ваш ход (Строка, столбец)
1 5
player 1's turn:
 O  .  .  .  O
 .  .  .  X  .
 .  .  X  .  .
 .  .  .  .  .
 .  .  .  .  .
player -1's turn:
 O  .  .  .  O
 .  .  .  X  .
 .  .  X  .  .
 .  X  .  .  .
 .  .  .  .  .
Введите ваш ход (Строка, столбец)
5 1
player 1's turn:
 O  .  .  .  O
 .  .  .  X  .
 .  .  X  .  .
 .  X  .  .  .
 O  .  .  .  .
player -1's turn:
 O  .  .  .  O
 .  .  .  X  .
 .  .  X  X  .
 .  X  .  .  .
 O  .  .  .  .
Введите ваш ход (Строка, столбец)
1 4
player 1's turn:
 O  .  .  O  O
 .  .  .  X  .
 .  .  X  X  .
 .  X  .  .  .
 O  .  .  .  .
player -1's turn:
 O  .  .  O  O
 .  .  

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
state2d = torch.tensor(np.array(
    [[[0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0]]]
)).to(device)

q_values = agent(state2d).squeeze(0).detach().cpu().numpy()
        # q_values[state2d != 0] = -float("Inf") # Маскирование
np.unravel_index(q_values.argmax(), q_values.shape)

(2, 2)

In [ ]:
q_values

array([[-0.05844196, -0.01638476, -0.03859643,  0.02083081, -0.04994318],
       [ 0.0068083 ,  0.03688353,  0.02646443,  0.04596754,  0.03287292],
       [-0.03988126,  0.05019872,  0.05487259,  0.03977484, -0.02035043],
       [ 0.0058784 ,  0.03290137,  0.04369787,  0.0137465 ,  0.04241205],
       [-0.03566115,  0.01458691, -0.01809969,  0.01231802, -0.05264549]],
      dtype=float32)